In [140]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import duckdb

base_dir = Path('c:\\Users\\henry\\OneDrive\\Personal Career\\Personal Projects\\GitHub\\Revenue-Sustainability-Analysis')
data_dir = Path(base_dir / 'Dataset')

## Define adopters vs. non adopters


In [141]:
pd.set_option('display.max_rows',10)

In [142]:
feats = pd.read_parquet(data_dir / 'feature_usage.parquet')

In [143]:
users = feats[feats['feature_name'] == 'feature_newai']

distinct_days = users.groupby('subscription_id')['usage_date'].nunique().reset_index(name='distinct_usage_days')
distinct_days["adoption_flag"] = (distinct_days['distinct_usage_days'] >= 3).astype(int)
distinct_days.groupby('distinct_usage_days')['subscription_id'].nunique().reset_index(name='num_users').sort_values('distinct_usage_days')

feats = feats.merge(distinct_days, on='subscription_id', how='left')
# feats['adoption_flag'] = feats['adoption_flag'].astype('Int64')

# Percentage of users who interacted with product who are adopters = 25.42%
distinct_days['adoption_flag'].mean()

0.2542105263157895

### Calculate pre-release metrics at the customer level

In [174]:
pre = feats[(feats['usage_date'] < '2023-06-12') & (feats['usage_date'] >= '2023-03-14')]

# Calculate number of active days
num_active_days = pre.groupby('subscription_id')['usage_date'].nunique().reset_index(name='distinct_active_days')

# Calculate avg gaps between days
pre['usage_date'] = pd.to_datetime(pre['usage_date'])
test = pre.sort_values(['subscription_id', 'usage_date']) \
    .groupby('subscription_id')['usage_date'] \
    .diff() \
    .dt.days

C:\Users\henry\AppData\Local\Temp\ipykernel_17968\3802385170.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre['usage_date'] = pd.to_datetime(pre['usage_date'])


In [ ]:
# pd.set_option('display.max_rows', 15)

In [175]:
test

3757      NaN
21020    63.0
16545     8.0
5749      NaN
2283      NaN
         ... 
35930     NaN
3795      NaN
15459     NaN
27357     NaN
29236     NaN
Name: usage_date, Length: 4478, dtype: float64

### Define pre-release window as 90 days prior to June 16, 2023 => March 14, 2023

In [ ]:
pd.read_parquet(data_dir / 'support_tickets.parquet')